In [491]:
import pandas as pd
import os
import re
from nltk.tokenize import WordPunctTokenizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from operator import getitem

In [492]:
#necessary corpora

stopwords_set = set(stopwords.words('english'))
required = {"needn", "doesn", "won", "shan't", "don", "ain", "not", "didn", "hadn", "haven't", "couldn't", "wasn't", "aren't", "isn", "needn't", "aren", "wouldn", "shouldn", "hasn't", "shan", "no", "wasn", "nor", "hasn", "mightn", "doesn't", "against", "wouldn't", "couldn", "hadn't", "isn't", "mustn", "don't", "weren't", "haven", "mustn't", "shouldn't", "weren", "won't", "mightn't"}
stopwords_set -= required
print(stopwords_set)
dictionary = set(nltk.corpus.words.words())

{'than', 'they', 'being', 'other', 'he', 't', 'myself', 'same', 'on', 'me', 'their', 'have', 'before', 'then', 'been', "it's", 'any', 'both', 'just', 'y', 'is', 'which', 'off', "you're", 've', 'yours', 'was', 'of', 'below', 'themselves', 'there', 'at', 'will', 'ma', 'above', 'you', 'now', 'll', 'we', 'very', 'over', 'these', 'it', 'until', 'as', 'that', 'himself', 'ours', 'once', "you've", 'did', 'under', 'o', 'after', 'down', 'be', 'from', 'when', 'about', "you'll", 'herself', 'through', 'his', 'ourselves', 'during', 'if', 'all', 'yourselves', 'my', 'where', 'our', 'those', 'out', 'the', 'your', 'so', 'doing', 'its', 'theirs', 'again', 're', 'with', 'and', 'him', "that'll", 'for', "should've", 'more', "didn't", 'yourself', 'here', 'up', 'by', 'while', 'in', 'd', 'into', 'further', 'too', 'do', 'them', 'does', 'but', 'most', 'what', 'having', 'her', 'why', 's', 'an', 'such', 'i', 'has', 'each', 'whom', 'few', 'she', "she's", 'because', 'between', 'how', 'a', 'or', 'some', 'had', "you'd

In [493]:
df = pd.read_csv("FinalTweetList_train.csv", header = None)

In [494]:
#upsample compliments to equalise classes

compliment = df[df[1]=='compliment']
displeasure = df[df[1]=='displeasure']
miscellaneous = df[df[1]=='miscellaneous']
size = min(len(displeasure), len(miscellaneous))
print(size)
compliment_extra = pd.DataFrame()
while(len(compliment) + len(compliment_extra) < size):
    compliment_extra = compliment_extra.append(compliment.sample(min(size - (len(compliment) + len(compliment_extra)),len(compliment))))
compliment = compliment_extra.append(compliment)

df = compliment.append([displeasure, miscellaneous])
df = df.reset_index(drop = True)
print(df)

# partially stratified upsample 80.5% train 74% test


4493
                                                       0              1
0      Very nice job by @KotakBankLtd .\nWhen #ATMs o...     compliment
1      @ICICIBank_Care Amazing work at @ICICIBank Dah...     compliment
2      @KotakBankLtd ur branch at vivek vihar was wel...     compliment
3      Thank you @ICICIBank_Care  @ICICIBank  for rev...     compliment
4      @ICICIBank @ICICIBank_Care i want to sincerely...     compliment
...                                                  ...            ...
14487  @AxisBank @HDFC_Bank @ICICIBank @IDBI_Bank @ID...  miscellaneous
14488  Latest status of ATMs in Jangpura, New Delhi. ...  miscellaneous
14489  I wonder why there is no queue @YESBANK Jalgao...  miscellaneous
14490  @ndtv In queue since 7 AM @YESBANK out of curr...  miscellaneous
14491  @ICICIBank @TheOfficialSBI @HDFC_Bank @AxisBan...  miscellaneous

[14492 rows x 2 columns]


In [495]:
#downsample displeasure and miscellaneous to equalise classes

'''
compliment = df[df[1]=='compliment']
displeasure = df[df[1]=='displeasure']
miscellaneous = df[df[1]=='miscellaneous']
_, displeasure = train_test_split(displeasure, test_size = len(compliment), random_state = 21)
_, miscellaneous = train_test_split(miscellaneous, test_size = len(compliment), random_state = 21)
df = compliment.append([displeasure, miscellaneous])
df = df.reset_index(drop = True)
print(df)
'''
#stratified downsample 88.5% train 67.4% test
#original 73.7% train 61.4% test

"\ncompliment = df[df[1]=='compliment']\ndispleasure = df[df[1]=='displeasure']\nmiscellaneous = df[df[1]=='miscellaneous']\n_, displeasure = train_test_split(displeasure, test_size = len(compliment), random_state = 21)\n_, miscellaneous = train_test_split(miscellaneous, test_size = len(compliment), random_state = 21)\ndf = compliment.append([displeasure, miscellaneous])\ndf = df.reset_index(drop = True)\nprint(df)\n"

In [496]:
#tweet cleaning

def processTweet(tweet):
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    #Convert to lower case
    tweet = tweet.lower()
    return tweet

for i in range(len(df[0])):
    df[0][i] = processTweet(df[0][i])

print(df)

                                                       0              1
0      very nice job by at_user . when atms of at_use...     compliment
1      at_user amazing work at at_user dahisar west b...     compliment
2      at_user ur branch at vivek vihar was well mana...     compliment
3      thank you at_user at_user for reversing the ch...     compliment
4      at_user at_user i want to sincerely appreciate...     compliment
...                                                  ...            ...
14487  at_user at_user at_user at_user at_user at_use...  miscellaneous
14488  latest status of atms in jangpura, new delhi. ...  miscellaneous
14489  i wonder why there is no queue at_user jalgaon...  miscellaneous
14490  at_user in queue since 7 am at_user out of cur...  miscellaneous
14491  at_user at_user at_user at_user at_user at_use...  miscellaneous

[14492 rows x 2 columns]


In [497]:
#Stopword removal, non-alphabet tokens removed, optional non-english word removal, Maxent feature extraction

#train, test = train_test_split(df, test_size = (500 if len(feats)>500 else 0.25), stratify = df[1], random_state = 21)
#train, test = train_test_split(df, test_size = (500 if len(feats)>500 else 0.25), random_state = 21)
#train = train.reset_index(drop = True)
#test = test.reset_index(drop = True)
train = df

def word_feats(words):
    #return {word:True for word in words if (word.isalpha())}    #downsample 87.5% train 66.8% test    upsample 75.7% train 71% test
    #return {word:True for word in words if (word.isalpha() and word in dictionary)}    #downsample 81.6% train 66% test    upsample 70.8% train 69% test
    return {word:True for word in words if (word.isalpha() and word not in stopwords_set)}    #downsample 88.5% train 67.4% test    upsample 80.5% train 74% test
    #return {word:True for word in words if (word.isalpha() and word not in stopwords_set and word in dictionary)}    #downsample 82.4% train 63.4% test    upsample 75.3% train 68% test
    
def extract_feats(dataframe):
    return [(word_feats(WordPunctTokenizer().tokenize(dataframe[0][i])), dataframe[1][i]) for i in range(len(dataframe[0]))]

train_set = extract_feats(train)

In [498]:
#Maxent train

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(train_set, algorithm, max_iter = 25)
classifier.show_most_informative_features(10)

  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.310
             2          -1.09806        0.830
             3          -1.09751        0.830
             4          -1.09696        0.830
             5          -1.09641        0.830
             6          -1.09587        0.830
             7          -1.09532        0.830
             8          -1.09477        0.830
             9          -1.09422        0.830
            10          -1.09368        0.830
            11          -1.09313        0.830
            12          -1.09259        0.830
            13          -1.09204        0.830
            14          -1.09150        0.830
            15          -1.09095        0.830
            16          -1.09041        0.830
            17          -1.08986        0.830
            18          -1.08932        0.830
            19          -1.08878        0.830
  

In [499]:
#Maxent testing on all banks and performance index calculation from classifier predictions

values = {
    "compliment": 1,
    "miscellaneous": 0.5,
    "displeasure": 0
}

results = {}

direc = os.fsencode('./test')
for File in os.listdir(direc):
    filename = os.fsdecode(File)
    try:
        test = pd.read_csv("./test/" + filename, header = None)
        for i in range(len(test[0])):
            test[0][i] = processTweet(test[0][i])
        test_set = extract_feats(test)
        y_true = list(test[1])
        y_pred = [classifier.classify(i[0]) for i in test_set]
        sum = 0
        for i in y_pred:
            sum += values[i]
        result = {}
        result['performance_index'] = sum * 100 / len(y_pred)
        result['classification_report'] = classification_report(y_true, y_pred)
        result['tweets'] = len(y_pred)
        results[filename[:-4]] = result
    except:
        pass

for i in sorted(results.items(), key = lambda x: ( -(getitem(x[1],'tweets')), -(getitem(x[1],'performance_index')) )):
    print((i[0] + ": ").rjust(40) + ("{:.2f}".format(i[1]['performance_index'])).rjust(6) + "% in " + str(i[1]['tweets']).rjust(4) + " tweets")
    print(i[1]['classification_report'])
    
#Toppers are:
    #1. Kotak Bank: 32.60%
    #2. Axis Bank: 30.60%
    #3. HDFC Bank: 27.70%
    #4. ICICI Bank: 24.30%
    #5. State Bank of India: 23.10%
#Almost correct order

                            Kotak Bank:  32.60% in  500 tweets
               precision    recall  f1-score   support

   compliment       0.86      0.90      0.88       113
  displeasure       0.73      0.92      0.81       232
miscellaneous       0.82      0.48      0.60       155

     accuracy                           0.78       500
    macro avg       0.81      0.77      0.77       500
 weighted avg       0.79      0.78      0.76       500

                             Axis Bank:  30.60% in  500 tweets
               precision    recall  f1-score   support

   compliment       0.73      0.85      0.79        41
  displeasure       0.79      0.95      0.86       201
miscellaneous       0.95      0.78      0.85       258

     accuracy                           0.85       500
    macro avg       0.82      0.86      0.83       500
 weighted avg       0.87      0.85      0.85       500

                             HDFC Bank:  27.70% in  500 tweets
               precision    recall 